In [2]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
# !pip install gmaps
!jupyter nbextension enable --py --sys-prefix gmaps
!jupyter nbextension enable --py gmaps

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time
import sys
%matplotlib inline
import scipy.stats as st
import pprint
# Import API key
with open('/Users/medinam/Downloads/config.py', 'r') as g_key:
    mykey=g_key.read()
    mykey=mykey.split('=')
    mykey=mykey[1].strip()   
g_key = mykey.strip('"')
#from api_keys import g_key
gmaps.configure(api_key=g_key)

#Data File path (CSV)
data = "./output_data/cities.csv"

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [3]:
# Study data files
RMS_path = "./RMS.csv"
# Read the data
Crime_df = pd.read_csv(RMS_path)
print(Crime_df)


       ARREST_DATE        DATE  YEAR     MONTH  WEEK SEX RACE ETHNIC  FBI  \
0         1/1/2016    1/1/2016  2016   JANUARY     1   F    W      N  NaN   
1         1/1/2016    1/1/2016  2016   JANUARY     1   M    W      H  NaN   
2         1/1/2016    1/1/2016  2016   JANUARY     1   M    W      H  NaN   
3         1/1/2016    1/1/2016  2016   JANUARY     1   M    W      H  NaN   
4         1/1/2016    1/1/2016  2016   JANUARY     1   F    W      H  NaN   
...            ...         ...   ...       ...   ...  ..  ...    ...  ...   
219219  12/31/2018  12/31/2018  2018  DECEMBER    53   M    B      N  NaN   
219220  12/31/2018  12/31/2018  2018  DECEMBER    53   M    B      N  NaN   
219221  12/31/2018  12/31/2018  2018  DECEMBER    53   M    W      H  NaN   
219222  12/31/2018  12/31/2018  2018  DECEMBER    53   M    W      H  NaN   
219223  12/31/2018  12/31/2018  2018  DECEMBER    53   M    W      H  NaN   

            SID  ... INVOLVEMENT  INCIDENT_NUM OFFENSE_CODE  \
0           

In [4]:
#Clean the Zipcode Data
Crime_df = Crime_df.dropna(subset=['ZIP'])
print(Crime_df)

       ARREST_DATE        DATE  YEAR     MONTH  WEEK SEX RACE ETHNIC  FBI  \
0         1/1/2016    1/1/2016  2016   JANUARY     1   F    W      N  NaN   
1         1/1/2016    1/1/2016  2016   JANUARY     1   M    W      H  NaN   
2         1/1/2016    1/1/2016  2016   JANUARY     1   M    W      H  NaN   
3         1/1/2016    1/1/2016  2016   JANUARY     1   M    W      H  NaN   
4         1/1/2016    1/1/2016  2016   JANUARY     1   F    W      H  NaN   
...            ...         ...   ...       ...   ...  ..  ...    ...  ...   
219219  12/31/2018  12/31/2018  2018  DECEMBER    53   M    B      N  NaN   
219220  12/31/2018  12/31/2018  2018  DECEMBER    53   M    B      N  NaN   
219221  12/31/2018  12/31/2018  2018  DECEMBER    53   M    W      H  NaN   
219222  12/31/2018  12/31/2018  2018  DECEMBER    53   M    W      H  NaN   
219223  12/31/2018  12/31/2018  2018  DECEMBER    53   M    W      H  NaN   

            SID  ... INVOLVEMENT  INCIDENT_NUM OFFENSE_CODE  \
0           

In [5]:
Crime_df=Crime_df.loc[(Crime_df["ZIP"]!="0") & (Crime_df["ZIP"]!="1") & (Crime_df["ZIP"]!="78") & 
                                (Crime_df["ZIP"]!="11111") & (Crime_df["ZIP"]!="6278") & (Crime_df["ZIP"]!="50701")
                               & (Crime_df["ZIP"]!="81507")]
print(Crime_df)

       ARREST_DATE        DATE  YEAR     MONTH  WEEK SEX RACE ETHNIC  FBI  \
0         1/1/2016    1/1/2016  2016   JANUARY     1   F    W      N  NaN   
1         1/1/2016    1/1/2016  2016   JANUARY     1   M    W      H  NaN   
2         1/1/2016    1/1/2016  2016   JANUARY     1   M    W      H  NaN   
3         1/1/2016    1/1/2016  2016   JANUARY     1   M    W      H  NaN   
4         1/1/2016    1/1/2016  2016   JANUARY     1   F    W      H  NaN   
...            ...         ...   ...       ...   ...  ..  ...    ...  ...   
219219  12/31/2018  12/31/2018  2018  DECEMBER    53   M    B      N  NaN   
219220  12/31/2018  12/31/2018  2018  DECEMBER    53   M    B      N  NaN   
219221  12/31/2018  12/31/2018  2018  DECEMBER    53   M    W      H  NaN   
219222  12/31/2018  12/31/2018  2018  DECEMBER    53   M    W      H  NaN   
219223  12/31/2018  12/31/2018  2018  DECEMBER    53   M    W      H  NaN   

            SID  ... INVOLVEMENT  INCIDENT_NUM OFFENSE_CODE  \
0           

In [6]:
# of zipcodes
Zipcodes_df = Crime_df.loc[:,["ZIP"]]
print(Zipcodes_df)
print(type(Zipcodes_df))
Zipcodes_df["ZIP"].unique()

          ZIP
0       78205
1       78201
2       78218
3       78201
4       78207
...       ...
219219  78220
219220  78220
219221  78222
219222  78257
219223  78228

[217446 rows x 1 columns]
<class 'pandas.core.frame.DataFrame'>


array(['78205', '78201', '78218', '78207', '78225', '78210', '78230',
       '78212', '78211', '78228', '78242', '78221', '78238', '78213',
       '78202', '78245', '78240', '78233', '78219', '78224', '78251',
       '78216', '78237', '78217', '78209', '78227', '78214', '78254',
       '78223', '78252', '78220', '78266', '78247', '78232', '78249',
       '78204', '78256', '78250', '78229', '78235', '78203', '78215',
       '78258', '78253', '78226', '78244', '78239', '78023', '78154',
       '78208', '78222', '78109', '78259', '78257', '78130', '78255',
       '78248', '78121', '78260', '78231', '78207-3170', '78236', '78241',
       '78155', '78070', '79502', '78233-6610', '78073', '78147', '78112',
       '78234', '78223-2099', '78016', '78000', '78264', '78148', '78263',
       '78056', '78006', '78205-2687', '78261', '78160', '78291', '78404',
       '78132', '78201-3847', '78108', '78026', '78009', '78238-4205',
       '78288', '78216-6902', '78054', '78163', '78200', '78015', '78

In [7]:
Num_of_Crimes_grp = Crime_df.groupby("ZIP")
Num_of_Crimes = Num_of_Crimes_grp["INCIDENT_NUM"].count()
Num_of_Crimes_2 = Num_of_Crimes.astype(float)
Num_of_Crimes_3 = pd.DataFrame(Num_of_Crimes_2)
print(type(Num_of_Crimes_3))
print(Num_of_Crimes_2)
print(Num_of_Crimes_3)

<class 'pandas.core.frame.DataFrame'>
ZIP
70205    10.0
74134     1.0
78000     1.0
78002     3.0
78006     4.0
         ... 
78288     8.0
78291     1.0
78404     1.0
78648     1.0
79502     3.0
Name: INCIDENT_NUM, Length: 106, dtype: float64
       INCIDENT_NUM
ZIP                
70205          10.0
74134           1.0
78000           1.0
78002           3.0
78006           4.0
...             ...
78288           8.0
78291           1.0
78404           1.0
78648           1.0
79502           3.0

[106 rows x 1 columns]


In [8]:
Zipcodes_df 

Zipcodes_df["Lat"] = ""
Zipcodes_df["Lon"] = ""

# Target zipcode
# Zipcodes_df_2["Zip"] = Zipcodes_df_2["ZIP"].astype(str)

print(Zipcodes_df.dtypes)
print(Zipcodes_df)

ZIP    object
Lat    object
Lon    object
dtype: object
          ZIP Lat Lon
0       78205        
1       78201        
2       78218        
3       78201        
4       78207        
...       ...  ..  ..
219219  78220        
219220  78220        
219221  78222        
219222  78257        
219223  78228        

[217446 rows x 3 columns]


In [10]:

for index,row in Zipcodes_df.iterrows():

    # get zipcode info from df
    Zipcode = row['ZIP']

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {Zipcode}.")
    # Build the endpoint URL
    target_url = ("https://maps.googleapis.com/maps/api/place/nearbysearch/json?query="+str(Zipcode)+"&key="+g_key)
    response = requests.get(target_url).json()
    pprint.pprint(response)
    
#     # extract results
#     results = response['results']
    
    
#     try:
#         print(f"The lat is: {results[0]['geometry']['location']['lat']}")
#         print(f"The lng is: {results[0]['geometry']['location']['lng']}")
        
#         Zipcodes_df.loc[index, "Lat"] = response['results'][0]['geometry']['location']['lat']
#         Zipcodes_df.loc[index, "Lon"] = response['results'][0]['geometry']['location']['lng']
        
#     except (KeyError, IndexError):
#         print("Missing field/result... skipping.")
        
#     print("------------")

# Zipcodes_df

Retrieving Results for Index 0: 78205.
{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
Retrieving Results for Index 1: 78201.
{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
Retrieving Results for Index 2: 78218.
{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
Retrieving Results for Index 3: 78201.
{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
Retrieving Results for Index 4: 78207.
{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
Retrieving Results for Index 5: 78207.
{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
Retrieving Resul

{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
Retrieving Results for Index 52: 78219.
{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
Retrieving Results for Index 53: 78219.
{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
Retrieving Results for Index 54: 78219.
{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
Retrieving Results for Index 55: 78219.
{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
Retrieving Results for Index 56: 78219.
{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
Retrieving Results for Index 57: 78219.
{'error_me

{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
Retrieving Results for Index 102: 78224.
{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
Retrieving Results for Index 103: 78224.
{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
Retrieving Results for Index 104: 78224.
{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
Retrieving Results for Index 105: 78224.
{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
Retrieving Results for Index 106: 78224.
{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
Retrieving Results for Index 107: 78224.
{'er

{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
Retrieving Results for Index 153: 78212.
{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
Retrieving Results for Index 154: 78223.
{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
Retrieving Results for Index 155: 78223.
{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
Retrieving Results for Index 156: 78216.
{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
Retrieving Results for Index 157: 78205.
{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
Retrieving Results for Index 158: 78205.
{'er

{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
Retrieving Results for Index 203: 78214.
{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
Retrieving Results for Index 204: 78228.
{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
Retrieving Results for Index 205: 78233.
{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
Retrieving Results for Index 206: 78233.
{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
Retrieving Results for Index 207: 78207.
{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
Retrieving Results for Index 208: 78228.
{'er

...

{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
Retrieving Results for Index 706: 78212.
{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
Retrieving Results for Index 707: 78207.
{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
Retrieving Results for Index 708: 78207.
{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
Retrieving Results for Index 709: 78207.
{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
Retrieving Results for Index 710: 78207.
{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
Retrieving Results for Index 711: 78207.
{'er

{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
Retrieving Results for Index 757: 78233.
{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
Retrieving Results for Index 758: 78205.
{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
Retrieving Results for Index 759: 78205.
{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
Retrieving Results for Index 760: 78205.
{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
Retrieving Results for Index 761: 78205.
{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
Retrieving Results for Index 762: 78218.
{'er

{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
Retrieving Results for Index 807: 78205.
{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
Retrieving Results for Index 808: 78154.
{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
Retrieving Results for Index 809: 78215.
{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
Retrieving Results for Index 810: 78221.
{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
Retrieving Results for Index 811: 78233.
{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
Retrieving Results for Index 812: 78218.
{'er

{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
Retrieving Results for Index 857: 78238.
{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
Retrieving Results for Index 858: 78214.
{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
Retrieving Results for Index 859: 78214.
{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
Retrieving Results for Index 860: 78214.
{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
Retrieving Results for Index 861: 78214.
{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
Retrieving Results for Index 862: 78214.
{'er

ConnectionError: ('Connection aborted.', TimeoutError(10060, 'A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond', None, 10060, None))

In [ ]:
Zipcodes_df_2["Lat"] = pd.to_numeric(Zipcodes_df_2["Lat"])
Zipcodes_df_2["Lon"] = pd.to_numeric(Zipcodes_df_2["Lon"])


locations = Zipcodes_df_2[["Lat", "Lon"]]
Crime = Num_of_Crimes_3
print(len(locations))
print(len(Crime))

In [ ]:
fig= gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=Crime, dissipating=False,
                                point_radius=5)

fig.add_layer(heat_layer)

fig